# Analysis Of TTC Ridership
** Group H - Final Project**<br>
** Data Source :** https://www.toronto.ca/city-government/data-research-maps/toronto-progress-portal/

** Feature Used**<br>
1. **sqlalchemy** package to save and read data from sqllite database
2. **numpy**<br>round(), mean(), arange(), array(), matrix()<br><br>
3. **panda**<br>concat(), read_sql_query(), read_excel(), read_csv(), autocorrelation_plot()
   pandas.plotting.table() draw data table in graph<br><br>
4. **dataframe**<br>set_index(), reset_index(), sort_index(), shift(), drop(), ffill(), fillna(0), astype(), to_sql(),
   min(), max(), sum(), mean(), groupby(), head(), tail(), transpose()<br><br>
5. **str**<br>format(), replace()<br><br>
6. **pyplot**<br>plot(), subplots(), acorr(), scatter() savefig()<br><br>
7. **sklearn**<br>LinearRegression(), fit(), score(), predict()<br><br>
8. **statsmodels**<br>ARIMA() model, fit(), forecast(), mean_squared_error()<br><br>
9. **language**<br>For loop, 
   immediate if conditions (True if len(df)>0 else False)

### Environment Setup

In [ ]:
# Import numpy and pandas 
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from pandas.plotting import table
from sqlalchemy import create_engine # database connection

# Import visualization libraries and set %mathplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline

### Variable Declaration

In [ ]:
# Global variables
dbName = 'sqlite:///TTCRidership.db'

# Table Names
tnPeakRiders = 'PeakRiders'
tnNonPeakRiders = 'NonPeakRiders'
tnRidershipAnalysis = 'RidershipAnalysis'
tnMonthlyRiders = 'MonthlyRiders'
tnRidershipRevenues = 'RidershipRevenues'

# Column names to fix for peak/non-peak riders data
colToFixRidesPeak = 'TTC Annual Passenger Rides Peak (000s)'
colToFixRidesNonPeak = 'TTC Annual Passenger Rides Non-Peak (000s)'

# New column names for peak/non-peak riders data
newColRidesPeak = 'PeakRides'
newColRidesNonPeak = 'NonPeakRides'

# Column name for monthly riders data
colMonthlyRidership = 'TTC Monthly Ridership'

# Source file names
srcFileRidesPeak = 'TTC Annual Passenger Rides Peak.csv'
srcFileRidesNonPeak = 'TTC Annual Passenger Rides Non-Peak.csv'
srcFileMonthlyRiders = 'TTC Monthly Ridership.csv'
srcFileRidershipRevenues = 'TTC Ridership Revenues.csv'

# Initializes database with filename TTCRidership.db in current directory
ttcDbConn = create_engine(dbName)

### Function Declaration

##### Database related functions

In [ ]:
# Function to read data from sqllite table
def ReadTableFromDB(tableName, dbConn):
    # read from sqllite db
    df = pd.read_sql_query("SELECT * FROM {tn};".format(tn=tableName), dbConn)
    
    #set the index back to first column of the table,
    df = df.set_index(df.columns[0])
    
    return df

# Function to check if table exist in the sqllite database
def IsTableExists(tableName, dbConn):
    sql="SELECT name FROM sqlite_master WHERE type='table' AND name='{tn}';".format(tn=tableName)
    df = pd.read_sql_query(sql, dbConn)
    exists = True if len(df)>0 else False
    return exists

# Function to write data frame in the sqllite database
def SaveDataframeInDB(dataFrame, tableName, dbConn):
    if dbConn is None:
        print('The dbConn is required to update database')
    else:
        dataFrame.to_sql(tableName, dbConn, if_exists='replace')


##### Data reading and cleaning functions

In [ ]:
# Function to fix the accumulated revenue values 
def FixAccumulatedMonthValues(data):
    previousColumn = ''
    for column in data.columns[:0:-1]:
        if (previousColumn==''):
            previousColumn = column

        if (previousColumn != column):
            data[previousColumn] = (data[previousColumn] - data[column]) / 1000000 # convert into millions

        previousColumn = column
    
    # convert the last month i.e Jan into convert into millions
    data[data.columns[1:2]] = data[data.columns[1:2]] / 1000000
    
    return data

In [ ]:
# function to split the string
def splitYear(row):
    return row.Year.split()[0]

# Function to calculate the growth percentage
def CalculateGrowth(data):
    diff = (data / data.shift(1) - 1) * 100
    return diff

# Function to fix the date data without the day
def FixDate(data):
    fixedData = pd.to_datetime(data, format='%b-%y', errors='ignore')
    return fixedData

In [ ]:
# Function to read ridership data, cleanup, fix and save to sqllite db for later use
def ReadAndFixTTCRidershipData(updateDB = False, dbConn = None):
    # Read excel file for Analysis of TTC ridership
    ridershipAnalysis = pd.read_excel('TTCRidership.xlsx', skiprows=5) # Skip first 5 rows with report headings    
    
    # Fix column names by providing the correct values to rename() function
    ridershipAnalysis = ridershipAnalysis.rename(columns=
                                             {
                                                 ' 2015 *': '2015', 
                                                 'Unnamed: 0': 'Ridership', 
                                                 '  FARE MEDIA': 'FareMedia'
                                             })
    
    # Delete rows starting from System Total onward using the index
    ridershipAnalysis = ridershipAnalysis.drop(ridershipAnalysis.index[46:52])
   
    # Find rows with sub-total in FareMedia values
    rowToDelete = ridershipAnalysis[ridershipAnalysis['FareMedia'].str.find('SUB-TOTAL')>0].index

    # Delete all rows with sub-total values
    ridershipAnalysis = ridershipAnalysis.drop(rowToDelete)

    # Find rows with system-total in FareMedia values
    rowToDelete = ridershipAnalysis[ridershipAnalysis['FareMedia'].str.find('SYSTEM TOTAL')>0].index

    # Delete all rows with system-total values
    ridershipAnalysis = ridershipAnalysis.drop(rowToDelete)    
    
    # Fix the ridership column values using forward fill method
    ridershipAnalysis['Ridership'] = ridershipAnalysis['Ridership'].ffill()
    
    # Find rows with the following categories values in FareMedia columns and
    # create a new attribute 'Passenger' of passenger types
    categoryList = ['ADULT', 'SENIOR/STUDENT', 'CHILDREN', 'OTHERS', 'BUS', 'RAIL', 'WEEKDAY', 'WEEKEND/HOLIDAY']
    ridershipAnalysis['Passenger'] = ridershipAnalysis[ridershipAnalysis.FareMedia.isin(categoryList)].FareMedia

    # Fix others category data
    categoryOther = 'DAY/VIST./OTHER'
    rowToUpdate = ridershipAnalysis[ridershipAnalysis.FareMedia.str.find(categoryOther)>0].index
    ridershipAnalysis.loc[rowToUpdate, 'Passenger'] = 'OTHERS'

    # Fix the Passenger column NaN values using forward fill method
    ridershipAnalysis['Passenger'] = ridershipAnalysis['Passenger'].ffill()

    # Find all category rows with Nan values and delete
    rowToDelete = ridershipAnalysis[ridershipAnalysis.FareMedia.isin(categoryList) & ridershipAnalysis['2016'].isnull()].index
    ridershipAnalysis = ridershipAnalysis.drop(rowToDelete)
    
    # Fill all NaN values with 0 and convert float values to int
    ridershipAnalysis.iloc[:,2:-1] = ridershipAnalysis.iloc[:,2:-1].fillna(0).astype(int)

    # Replace trailing spaces in FareMedia values
    ridershipAnalysis['FareMedia'] = ridershipAnalysis['FareMedia'].str.strip(' ')
    
    # Reset the index
    ridershipAnalysis = ridershipAnalysis.reset_index(drop=True)

    # Save to sqllite db if updateDB is True
    if updateDB:
        SaveDataframeInDB(ridershipAnalysis, tnRidershipAnalysis, dbConn)

    return ridershipAnalysis

# Function to get the ridership data
def GetRidershipAnalysisData():
    # Check if data is already cleaned and saved into db
    if IsTableExists(tnRidershipAnalysis, ttcDbConn):
        # Read ridership analysis data from db
        result = ReadTableFromDB(tnRidershipAnalysis, ttcDbConn)
    else :
        # Call function to read TTC ridership source file, clean the data and save into the db
        result = ReadAndFixTTCRidershipData(True, ttcDbConn)
    return result


In [ ]:
# Function to read peak/non-peak ridership data, cleanup, fix and save to sqllite db for later use
def ReadAndFixPeakNonPeakRidershipData(sourceFileName, tableName, oldColName, newColName, updateDB = False, dbConn = None):
    # Read TTC Annual Passenger Rides Non-Peak
    #nonPeakRiders = pd.read_csv('TTC Annual Passenger Rides Non-Peak.csv')
    ridersData = pd.read_csv(sourceFileName)

    # fix year column by splitting
    ridersData.Year = ridersData.apply(splitYear, axis=1)

    # set the index to year of ridership
    ridersData = ridersData.set_index('Year')
    
    # Fix column names by providing the correct values to rename() function
    ridersData = ridersData.rename(
        columns= { '{ocn}'.format(ocn=oldColName) : '{ncn}'.format(ncn=newColName) }
    )
    
    # create new attribute Growth by using the shifting the data
    ridersData['Growth'] = CalculateGrowth(ridersData[newColName])

    # fill the empty year
    ridersData.fillna(0)

    # sort the year in descending and get the top 5 or pass number to head() for getting x years of data
    ridersData = ridersData.sort_index(ascending=False) # .head().sort_index()

    # Save to sqllite db if updateDB is True
    if updateDB:
        SaveDataframeInDB(ridersData, tableName, dbConn)
            
    return ridersData

# Function to get the ridership data
def GetNonPeakRidersData():
    # Check if data is already cleaned and saved into db
    if IsTableExists(tnNonPeakRiders, ttcDbConn):
        # Read ridership analysis data from db
        result = ReadTableFromDB(tnNonPeakRiders, ttcDbConn)
    else:
        # Call function to read TTC ridership source file, clean the data and save into the db
        result = ReadAndFixPeakNonPeakRidershipData(
            srcFileRidesNonPeak,
            tnNonPeakRiders, 
            colToFixRidesNonPeak, 
            newColRidesNonPeak,
            True, 
            ttcDbConn)
        
    return result

# Function to get the ridership data
def GetPeakRidersData():
    # Check if data is already cleaned and saved into db
    if IsTableExists(tnPeakRiders, ttcDbConn):
        # Read peak riders  data from db
        result = ReadTableFromDB(tnPeakRiders, ttcDbConn)
    else:
        # Call function to read peak/non-peak riders source file, clean the data and save into the db
        result = ReadAndFixPeakNonPeakRidershipData(
            srcFileRidesPeak,
            tnPeakRiders, 
            colToFixRidesPeak, 
            newColRidesPeak,
            True, 
            ttcDbConn)
        
    return result

In [ ]:
# Function to read monthly ridership data, cleanup, fix and save to sqllite db for later use
def ReadAndFixMonthlyRidershipData(sourceFileName, tableName, columnName, updateDB = False, dbConn = None):
    # Read TTC Monthly Passenger Rides
    monthlyRiders = pd.read_csv(sourceFileName, parse_dates=['Year'])

    # set the index to year of ridership
    monthlyRiders = monthlyRiders.set_index('Year')

    # create new attribute Growth by using the shifting the data
    monthlyRiders['Growth'] = CalculateGrowth(monthlyRiders[columnName])

    # fill the empty year
    monthlyRiders.fillna(0)

    # Find rows for 2017
    rowToDelete = monthlyRiders[monthlyRiders.index.str.endswith('17')].index

    # Delete all rows for 2017
    monthlyRiders = monthlyRiders.drop(rowToDelete)
    
    # Save to sqllite db if updateDB is True
    if updateDB:
        SaveDataframeInDB(monthlyRiders, tableName, dbConn)
            
    return monthlyRiders

# Function to get the ridership data
def GetMonthlyRidersData():
    # Check if data is already cleaned and saved into db
    if IsTableExists(tnMonthlyRiders, ttcDbConn):
        # Read ridership analysis data from db
        result = ReadTableFromDB(tnMonthlyRiders, ttcDbConn)
    else:
        # Call function to read TTC monthly ridership source file, clean the data and save into the db
        result = ReadAndFixMonthlyRidershipData(
            srcFileMonthlyRiders,
            tnMonthlyRiders,
            colMonthlyRidership,
            True, 
            ttcDbConn)
        
    return result


In [ ]:
# Function to read ridership revenue data, cleanup, fix and save to sqllite db for later use
def ReadAndFixRidersRevenueData(sourceFileName, tableName, updateDB = False, dbConn = None):
    # Read TTC Ridership Revenues data
    ridersRevenue = pd.read_csv(sourceFileName)
    
    # Fix the dollar amount and convert into int, by replacing $ and ,
    # create a list of columns and put them in a list
    cols = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # pass them to replace(), specifying each char and it's replacement:
    ridersRevenue[cols] = ridersRevenue[cols].replace({'\$': '', ',': ''}, regex=True)

    # fill NaN value
    ridersRevenue[cols] = ridersRevenue[cols].fillna(0)

    # Convert data to integer
    ridersRevenue[cols] = ridersRevenue[cols].astype(int)

    # fix year column by splitting and set the index
    ridersRevenue = ridersRevenue.set_index(ridersRevenue.apply(splitYear, axis=1))

    # Call function to update data
    FixAccumulatedMonthValues(ridersRevenue)

    ridersRevenue = ridersRevenue[ridersRevenue.index != '2017']
    
    # Save to sqllite db if updateDB is True
    if updateDB:
        SaveDataframeInDB(ridersRevenue, tableName, dbConn)
            
    return ridersRevenue

# Function to get the ridership data
def GetRidersRevenueData():
    # Check if data is already cleaned and saved into db
    if IsTableExists(tnRidershipRevenues, ttcDbConn):
        # Read ridership analysis data from db
        result = ReadTableFromDB(tnRidershipRevenues, ttcDbConn)
    else:
        # Call function to read TTC ridership revenue source file, clean the data and save into the db
        result = ReadAndFixRidersRevenueData(
            srcFileRidershipRevenues,
            tnRidershipRevenues,
            True, 
            ttcDbConn)
        
    return result

##### Graph related function

In [ ]:
# Function to save the graph
def SaveGraph(title, fig):
    # Cleans the title to use as filename for graph to save
    fileName = title.replace("/", " ").replace(" ", "")
    # Save the graph as png file, used bbox_inches='tight' to fit the whole chart when saving
    fig.savefig("{fn}.png".format(fn=fileName), dpi=fig.dpi, bbox_inches='tight')
   
    
# Function to set explode value for pie slice
def isExplode(x):
    if x['Explode'] == True:
        return 0.1
    else:
        return 0


# Function to draw pie chart and explode the slice with max size
def DrawPieChart(data, title, excludeLessThan = True, legend = ''):
    # Calculate the pie size and create new attribute
    data['Size'] = data / data.sum() * 100

    # Create new attribute to set the explode slice
    data['Explode'] = data['Size'].max() == data['Size']    
    data['Explode'] = data.apply(isExplode, axis=1)
                                       
    # Exclude slice values less than 1%
    if excludeLessThan == True:
        data = data[data['Size']>1]
    else:
        data = data[data['Size']>0]

    # Set the label, slice percent and size to explode
    labels = data.index
    sizes = data['Size']
    explode = data['Explode']  # only "explode" the slice with max size

    if legend == '':
        legend = labels
        
    fig, ax = plt.subplots(figsize=(15,8))
    ax.pie(sizes, explode=explode, autopct='%1.1f%%', shadow=True, startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.legend(legend, loc="upper right")
    plt.title(title, fontsize=20)
    plt.tight_layout()
    plt.show()
    print(data)
    
    # Call function to save the graph as png file
    SaveGraph(title, fig)
    

# Function to plot a chart of type kind, default is line
def DrawChart(data, title, yLabel, kind='line'):
    
    # Define subplots to draw line chart and the summary of the data we have
    fig, ax = plt.subplots(1, 1)

    data.plot(kind=kind, ax=ax, figsize=(15,8))
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
    plt.title(title, fontsize=20)
    plt.ylabel(yLabel)
    plt.show()
    print(data)

    # Call function to save the graph as png file
    SaveGraph(title, fig)
    
def DrawRidershipTrend(data):
    # Get total ridership from all years 
    totalRidership =pd.DataFrame({'Total Ridership': data[['WEEKDAY','WEEKEND/HOLIDAY']].transpose().sum()/1000})

    # Define subplots to draw line chart and the summary of the data we have
    fig, ax = plt.subplots(1, 1)

    # Draw the table describing the data we have for ridership
    table(ax, np.round(totalRidership.describe(), 2), loc='lower right', colWidths=[0.2, 0.2, 0.2]) 

    # Format title for the plot using min and max for index
    title = "Ridership Trend From {sy} To {ey}".format(sy=totalRidership.index.min(), ey=totalRidership.index.max())

    # Plot line chart for the ridership
    totalRidership.plot(title=title, ax=ax, legend=None, figsize=(15,4), linestyle='-', marker='o' )
    plt.ylabel("Total Ridership in Millions")
    # Hack to draw all year in the graph :)
    bar = totalRidership/1000
    bar.plot(kind='bar', ax=ax)

    # Call function to save the graph as png file
    SaveGraph(title, fig)
    
    return totalRidership

# Function to set the label of subplots
def label(ax, string):
    ax.annotate(string, (1, 1), xytext=(-8, -4), ha='right', va='top', size=14, 
                xycoords='axes fraction', textcoords='offset points')    

# Function to draw the autocorrelation graph
def DrawAutoCorrelation(data, title):
    
    fig, axes = plt.subplots(nrows=3, figsize=(12, 12))
    
    # using tight layout so there is no space between title and sub plots
    fig.tight_layout()
    
    # set the title for graph
    fig.suptitle(title, fontsize=16)
    
    label(axes[0], 'Trend')
    axes[0].plot(data)    
    
    lags, c, line, b = axes[1].acorr(data, maxlags=data.size-1)
    label(axes[1], 'Matplotlib Autocorrelation %.2f' % np.mean(c))
    
    label(axes[2], 'Pandas Autocorrelation')
    pd.plotting.autocorrelation_plot(data, ax=axes[2])    
    plt.show()    

### Read and process ridership data

In [ ]:
# Call function to get the ridership data either from db or source file
ridershipAnalysis = GetRidershipAnalysisData()

# Sort the data by columns
ridershipAnalysisSorted = ridershipAnalysis[ridershipAnalysis.columns.sort_values()]

# Groups the data by Passenger and Fare Type and also transpose
fixedRidershipData = ridershipAnalysisSorted.groupby(['Passenger', 'FareMedia']).mean().transpose()

## Total Ridership Trend

In [ ]:
# Call function to draw total ridership graph
totalRidership = DrawRidershipTrend(fixedRidershipData)

## Ridership proportion by passenger type

In [ ]:
# Draw pie chart for Adult media type for 2016
fareMediaForAdult = fixedRidershipData['ADULT'].tail(1).transpose()
DrawPieChart(fareMediaForAdult, 'Ridership of Adult Passengers in 2016')

In [ ]:
# Draw pie chart for Children media type for 2016
fareMediaForChildren = fixedRidershipData['CHILDREN'].tail(1).transpose()
DrawPieChart(fareMediaForChildren, 'Ridership of Children Passengers in 2016', False)

In [ ]:
# Draw pie chart for Senior/Student media type for 2016
fareMediaForSeniorStudent = fixedRidershipData['SENIOR/STUDENT'].tail(1).transpose()
DrawPieChart(fareMediaForSeniorStudent, 'Ridership of Senior/Student Passengers in 2016', False)

In [ ]:
# Draw pie chart for Senior/Student media type for 2016
fareMediaForSeniorStudent = fixedRidershipData['RAIL'].tail(1).transpose()
DrawPieChart(fareMediaForSeniorStudent, 'Ridership of Rail Passengers in 2016', False)

In [ ]:
# Draw pie chart for Other media type for 2016
fareMediaForSeniorStudent = fixedRidershipData['OTHERS'].tail(1).transpose()
DrawPieChart(fareMediaForSeniorStudent, 'Ridership of Others Passengers in 2016', False)

In [ ]:
# Draw pie chart for Other media type for 2016
fareMediaForSeniorStudent = fixedRidershipData[['WEEKDAY', 'WEEKEND/HOLIDAY']].tail(1).transpose()
DrawPieChart(fareMediaForSeniorStudent, 'Weekly Ridership in 2016', False, ['WEEKDAY', 'WEEKEND/HOLIDAY'])

## Yearly Ridership by Bus and Rail

In [ ]:
# Draw line chart for BUS and RAIL media type
# This should include all modes of transport
DrawChart(fixedRidershipData[['BUS','RAIL']].tail()/1000, "Bus/Rail ridership", "Ridership in Millions")

## Yearly Ridership by Media Type

In [ ]:
DrawChart(fixedRidershipData['ADULT'].tail()/1000, "ADULT ridership", "Ridership in Millions")

In [ ]:
DrawChart(fixedRidershipData['CHILDREN'].tail()/1000, "CHILDREN ridership", "Ridership in Millions")

In [ ]:
DrawChart(fixedRidershipData['SENIOR/STUDENT'].tail()/1000, "SENIOR/STUDENT ridership", "Ridership in Millions")

In [ ]:
DrawChart(fixedRidershipData[['WEEKDAY','WEEKEND/HOLIDAY']]/1000, "Weekday vs Weekend Ridership", "Ridership in Millions")

## Yearly weekday and weekend ridership ratio

In [ ]:
fixedRidershipData['Ratio'] = fixedRidershipData['WEEKDAY']['WEEKDAY'] / fixedRidershipData['WEEKEND/HOLIDAY']['WEEKEND/HOLIDAY']
DrawChart(fixedRidershipData[['Ratio']], "Ratio of Weekday to Weekend Ridership", "%")

# Autocorrelation for different parameters

In [ ]:
# Group the data by Passenger  and sum the value
ridershipDataForCorr = ridershipAnalysisSorted.groupby(['Passenger']).sum().transpose()

# We get the following groups
# ADULT, SENIOR/STUDENT, CHILDREN, OTHERS, WEEKDAY, WEEKEND/HOLIDAY, BUS, RAIL

In [ ]:
# Draw graph for Adult ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['ADULT']/1000, 'Adults Ridership')

In [ ]:
# Draw graph for SENIOR/STUDENT ridership
DrawAutoCorrelation(ridershipDataForCorr['SENIOR/STUDENT']/1000, 'Senior/Student Ridership')

In [ ]:
# Draw graph for CHILDREN ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['CHILDREN']/1000, 'Children Ridership')

In [ ]:
# Draw graph for OTHERS ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['OTHERS']/1000, 'Others Ridership')

In [ ]:
# Draw graph for WEEKDAY ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['WEEKDAY']/1000, 'Weekday Ridership')

In [ ]:
# Draw graph for WEEKEND/HOLIDAY ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['WEEKEND/HOLIDAY']/1000, 'Weekend/Holiday Ridership')

In [ ]:
# Draw graph for BUS ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['BUS']/1000, 'Bus Ridership')

In [ ]:
# Draw graph for Rail ridership also convert the values into thousands
DrawAutoCorrelation(ridershipDataForCorr['RAIL']/1000, 'Rail Ridership')

# Annual Peak and Non-Peak Ridership

### Read and process non peak ridership data

In [ ]:
# Call function to get the non peak ridership data either from db or source file
nonPeakRiders = GetNonPeakRidersData()

# get the top 5 or pass number to head() for getting x years of data
topNonPeakRiders = nonPeakRiders.head().sort_index()

DrawChart(topNonPeakRiders.Growth, "Annual Ridership Non-Peak", "Ridership Growth Rate")

### Read and process peak ridership data

In [ ]:
# Call function to get the non peak ridership data either from db or source file
peakRiders = GetPeakRidersData()

# get the top 5 or pass number to head() for getting x years of data
topPeakRiders = peakRiders.head().sort_index()

DrawChart(topPeakRiders.Growth, "Annual Passenger Rides Peak", "Ridership Growth Rate")

### Comparing Peak and Non-Peak Ridership

In [ ]:
# Concat peak and non peak riders data frame
peakNopeak = pd.concat([topPeakRiders, topNonPeakRiders], axis =1)
peakNopeak

In [ ]:
# Rename Growth column names
peakNopeak.columns.values[1] = "Peak Growth"
peakNopeak.columns.values[3] = "Non-Peak Growth"

In [ ]:
# Draw chart for 
DrawChart(peakNopeak[['Peak Growth', 'Non-Peak Growth']], 
          "Annual Riders Growth Peak & Non-Peak", 
          "%")

In [ ]:
DrawChart(peakNopeak[[newColRidesPeak, newColRidesNonPeak]]/1000, 
          "Annual Riders Peak & Non-Peak", 
          "Ridership in Millions")

# Monthly Passenger Ridership

In [ ]:
# Read monthly riders data
monthlyRiders = GetMonthlyRidersData()

# Get the last 5 months or pass number to tail() for getting x months of data
topMonthlyRiders = monthlyRiders.tail(24)

In [ ]:
DrawChart(topMonthlyRiders.Growth, "Monthly Passenger Rides Peak", "Ridership Growth Rate")

# Ridership Revenues

In [ ]:
# Read riders revenue data excluding 2017
ridersRevenueWOTarget = GetRidersRevenueData()
ridersRevenueWOTarget

In [ ]:
DrawChart(ridersRevenueWOTarget[ridersRevenueWOTarget.columns[2:]].tail(), 
          "Monthly Revenue", "Amount in millions", 'bar')

In [ ]:
monthly = ridersRevenueWOTarget[ridersRevenueWOTarget.columns[1:]].transpose()
monthly[monthly.columns[5:]]

In [ ]:
DrawChart(monthly[monthly.columns[5:]], 
          "Monthly Revenue comparison with previous years", "Amount in millions", 'bar')

# Regression for Ridership and Revenue trend

In [ ]:
# Arrange data for linear regression
# Sum the transpose ridership and revenue data by year
revenue = ridersRevenueWOTarget[ridersRevenueWOTarget.columns[1:]].transpose().sum()
ridership = fixedRidershipData[['WEEKDAY','WEEKEND/HOLIDAY']].tail(10).transpose().sum()/1000

In [ ]:
# Create data frame for regression test
data = {'Revenue': revenue / 100, 'Ridership': ridership / 100}
ridershipRevenueData = pd.DataFrame(data)

In [ ]:
# Prepare data to use with statsmodels OLS model
X = ridershipRevenueData['Ridership']
y = ridershipRevenueData['Revenue']
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

In [ ]:
# Prepare data to use with sklearn linear regression
lr = LinearRegression()
data = np.matrix( ridershipRevenueData )
X, Y = data[:,1], data[:,0]
lr.fit(X, Y)
score = '{0:.3f}'.format( lr.score(X, Y) )
score

In [ ]:
# Draw scatter graph and plot predicted test data - alpha=0.5, 
fig, axScatter = plt.subplots(figsize=(15, 8))
axScatter.scatter(ridershipRevenueData.Ridership, ridershipRevenueData.Revenue, 
                  marker='o', edgecolor='b', facecolor='b', alpha=0.4, s=80, label='Ridership and Revenue')

test = np.arange( 3.75, 5.75, 0.1 )
test = np.array( np.matrix( test ).T )
plt.plot( test, lr.predict( test ), 'g--')

plt.xlabel('Ridership')
plt.ylabel('Revenue')
plt.axis([4.5, 5.5,7.0,12.0])
plt.title('Regression of Revenue from Ridership') ;
plt.grid() ;
plt.legend(loc='upper left')
plt.savefig('Ridership_v_Revenue.png')
plt.show()

# Regression Using statsmodels ARIMA model

In [ ]:
# Autoregressive integrated moving average
# The ARIMA model can be used to forecast future time steps
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

series = totalRidership['Total Ridership']
X = series.values

# Calculate the size for breaking the ridership data into model and test data
size = int(len(X) * 0.9)

# breaking data in to model and test data
train, test = X[0:size], X[size:len(X)]

# declare a variable to hold the training model data
history = [x for x in train]

# variable to keep the predicted data
predictions = list()

# loop through test data and predict the trend
for t in range(len(test)):
    
    # Fit an ARIMA(5,1,0) model. 
    # This sets the lag value to 5 for autoregression, 
    # uses a difference order of 1 to make the time series stationary, 
    # and uses a moving average model of 0.    
    model = ARIMA(history, order=(5,1,0)) 
    model_fit = model.fit(disp=0)
    
    # get the output of forcast()
    output = model_fit.forecast()
    yhat = output[0]
    
    # add to prediction list
    predictions.append(yhat)
    
    # add test data into history list
    obs = test[t]
    history.append(obs)
    
    # print predicated and expected value
    #print('predicted=%f, expected=%f' % (yhat, obs))
    
# Calculate a final mean squared error score (MSE) for the predictions, 
# providing a point of comparison for other ARIMA configurations.
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

# Prepare data to plot chart
predicted = [x[0] for x in predictions]
data = {'Expected': test, 'Predicted': predicted}
predictionData = pd.DataFrame(data)

#Create subplot to plot line chart
fig, axScatter = plt.subplots(figsize=(15, 8))
axScatter.plot(test, 'g-', label='Expected')
axScatter.plot(predicted, 'r--', label='Predicted')
plt.xlabel('Period')
plt.ylabel('Ridership')
plt.title('Autoregressive integrated moving average of Ridership', fontsize=20) ;
plt.grid() ;
plt.legend(loc='upper left', fontsize=20)
plt.savefig('ARIMARidership.png')
plt.show()